In [35]:
# !sudo pip3 install keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow opencv--python sklearn optuna scikit-image ray ray[tune]

In [36]:
# !sudo pip3 install optkeras

In [38]:
import pickle
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import asarray
from numpy import save

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

model= load_model("model.h5")
train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255


from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna

optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')

def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)

    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value

ok.optimize(objective, timeout = 3*60)
import pandas as pd
""" Show Results """
print('Best trial number: ', ok.best_trial.number)
print('Best value:', ok.best_trial.value)
print('Best parameters: \n', ok.best_trial.params)

print("Best parameters (retrieved directly from Optuna)", ok.study.best_trial.params)

""" Check the Optuna CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.optuna_log_file_path, '\n')
display(pd.read_csv(ok.optuna_log_file_path))

""" Check the Keras CSV log file """
pd.options.display.max_rows = 8 # limit rows to display
print('Data Frame read from', ok.keras_log_file_path, '\n')
display(pd.read_csv(ok.keras_log_file_path))

put_csv = pd.read_csv(ok.keras_log_file_path)
put_csv.to_csv("Optuna_log_file.csv")

[2020-08-17 17:50:47.214889] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:107: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


[2020-08-17 17:50:48.113457]  (None) 


[I 2020-08-17 17:54:12,184] Trial 0 finished with value: 0.8333333333333334 and parameters: {'activation': 'linear'}. Best is trial 0 with value: 0.8333333333333334.


[2020-08-17 17:54:12.247349] Trial#: 0, value: 8.333333e-01| Best trial#: 0, value: 8.333333e-01, params: {'activation': 'linear'}
Best trial number:  0
Best value: 0.8333333333333334
Best parameters: 
 {'activation': 'linear'}
Best parameters (retrieved directly from Optuna) {'activation': 'linear'}
Data Frame read from CatsAndDogs_Simple_Optuna.csv 



,n_u_m_b_e_r,v_a_l_u_e,d_a_t_e_t_i_m_e___s_t_a_r_t,d_a_t_e_t_i_m_e___c_o_m_p_l_e_t_e,d_u_r_a_t_i_o_n,p_a_r_a_m_s___a_c_t_i_v_a_t_i_o_n,u_s_e_r___a_t_t_r_s_____D_a_t_e_t_i_m_e___e_p_o_c_h___b_e_g_i_n,u_s_e_r___a_t_t_r_s_____D_a_t_e_t_i_m_e___e_p_o_c_h___e_n_d,u_s_e_r___a_t_t_r_s_____T_r_i_a_l___n_u_m,u_s_e_r___a_t_t_r_s___a_c_c,u_s_e_r___a_t_t_r_s___l_o_s_s,u_s_e_r___a_t_t_r_s___v_a_l___a_c_c,u_s_e_r___a_t_t_r_s___v_a_l___l_o_s_s,s_t_a_t_e
0,0,0.833333,2020-08-17 17:50:47.222921,2020-08-17 17:54:12.184589,0 days 00:03:24.961668,linear,2020-08-17 17:53:04.385203,2020-08-17 17:54:11.698978,0,0.826389,0.361785,0.833333,0.366107,COMPLETE


Data Frame read from CatsAndDogs_Simple_Keras.csv 



,epoch,_Datetime_epoch_begin,_Datetime_epoch_end,_Trial_num,acc,loss,val_acc,val_loss
0,0,2020-08-17 11:55:30.050279,2020-08-17 11:56:34.176302,0,0.664352,0.681994,0.791667,0.458063
1,0,2020-08-17 11:57:51.630279,2020-08-17 11:58:55.985174,1,0.645833,0.687976,0.750000,0.502284
2,1,2020-08-17 11:58:56.466416,2020-08-17 12:00:00.389093,1,0.793981,0.443230,0.812500,0.398350
3,2,2020-08-17 12:00:00.764620,2020-08-17 12:01:11.526948,1,0.837963,0.343374,0.770833,0.394485
...,...,...,...,...,...,...,...,...
6,2,2020-08-17 13:37:28.717549,2020-08-17 13:38:28.443367,4,0.835648,0.356857,0.812500,0.351737
7,0,2020-08-17 17:50:48.299076,2020-08-17 17:51:58.125476,0,0.648148,1.204050,0.812500,0.466312
8,1,2020-08-17 17:51:58.595093,2020-08-17 17:53:04.384450,0,0.770833,0.474373,0.708333,0.454805
9,2,2020-08-17 17:53:04.385203,2020-08-17 17:54:11.698978,0,0.826389,0.361785,0.833333,0.366107


In [ ]:
### -------------------------CHECKPOINTS--------------------------------
def extract_checkpoints(path, output_directory):
    
    try:
        tar = tarfile.open(path)
        tar.extractall(path=output_directory)
        return tar.getnames()
    except (tarfile.ReadError, IOError):
        return []


def save_checkpoint(model,optimizer,current_metrics, checkpoints_files, losses_dict):
    
    filename='checkpoint_' + str(current_metrics[0]) +'.pth'
    checkpoint = { 'curr_epoch': current_metrics[0],
                   'model_state_dict' : model.state_dict(),
                   'optimizer_state_dict': optimizer.state_dict(),
                   'checkpoints_list': checkpoints_files,
                   'train_loss': current_metrics[1],
                   'test_loss': current_metrics[2],
                   'test_accuracy': current_metrics[3],
                   'losses_dict': losses_dict
    }
    checkpoints_files.append(filename)
    torch.save(checkpoint, filename)
    return checkpoints_files


def load_checkpoint(model,optimizer,filepath):
    
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['curr_epoch'] + 1
    losses_dict = checkpoint['losses_dict'] 

    return model, start_epoch, optimizer, losses_dict


def checkpoints_tar(checkpoints_files, tar_file):
    
    tar = tarfile.open(tar_file,"w:gz")
    for name in checkpoints_files:
        tar.add(name)
    tar.close()